In [1]:
'''
Created on Aug 9, 2016
Keras Implementation of Neural Matrix Factorization (NeuMF) recommender model in:
He Xiangnan et al. Neural Collaborative Filtering. In WWW 2017.  

@author: Xiangnan He (xiangnanhe@gmail.com)
'''
import numpy as np

#import os
#os.environ['KERAS_BACKEND'] = 'theano'

import theano
import theano.tensor as T
import keras
from keras import backend as K
from keras import initializers
from keras.regularizers import l1, l2, l1_l2
from keras.models import Sequential, Model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, Concatenate, Reshape, Multiply, Flatten, Dropout
from keras.optimizers import Adagrad, Adam, SGD, RMSprop
from evaluate import evaluate_model
from Dataset import Dataset
from time import time
import sys
import GMF, MLP
import argparse

#################### Arguments ####################
def parse_args():
    parser = argparse.ArgumentParser(description="Run NeuMF.")
    parser.add_argument('--path', nargs='?', default='Data/',
                        help='Input data path.')
    parser.add_argument('--dataset', nargs='?', default='ml-1m',
                        help='Choose a dataset.')
    parser.add_argument('--epochs', type=int, default=100,
                        help='Number of epochs.')
    parser.add_argument('--batch_size', type=int, default=256,
                        help='Batch size.')
    parser.add_argument('--num_factors', type=int, default=8,
                        help='Embedding size of MF model.')
    parser.add_argument('--layers', nargs='?', default='[64,32,16,8]',
                        help="MLP layers. Note that the first layer is the concatenation of user and item embeddings. So layers[0]/2 is the embedding size.")
    parser.add_argument('--reg_mf', type=float, default=0,
                        help='Regularization for MF embeddings.')                    
    parser.add_argument('--reg_layers', nargs='?', default='[0,0,0,0]',
                        help="Regularization for each MLP layer. reg_layers[0] is the regularization for embeddings.")
    parser.add_argument('--num_neg', type=int, default=4,
                        help='Number of negative instances to pair with a positive instance.')
    parser.add_argument('--lr', type=float, default=0.001,
                        help='Learning rate.')
    parser.add_argument('--learner', nargs='?', default='adam',
                        help='Specify an optimizer: adagrad, adam, rmsprop, sgd')
    parser.add_argument('--verbose', type=int, default=1,
                        help='Show performance per X iterations')
    parser.add_argument('--out', type=int, default=1,
                        help='Whether to save the trained model.')
    parser.add_argument('--mf_pretrain', nargs='?', default='',
                        help='Specify the pretrain model file for MF part. If empty, no pretrain will be used')
    parser.add_argument('--mlp_pretrain', nargs='?', default='',
                        help='Specify the pretrain model file for MLP part. If empty, no pretrain will be used')
    return parser.parse_args()

def init_normal(shape, dtype = None, name=None):
    # RandomNormal does not have scale and name
    return initializers.RandomNormal(shape)  # scale=0.01, , name=name

def get_model(num_users, num_items, mf_dim=10, layers=[10], reg_layers=[0], reg_mf=0):
    assert len(layers) == len(reg_layers)
    num_layer = len(layers) #Number of layers in the MLP
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    
    print('Input layer: ',user_input.shape,user_input.dtype)
    
    # Embedding layer
    """MF_Embedding_User = Embedding(input_dim = num_users, output_dim = mf_dim, name = 'mf_embedding_user',
                                  init = init_normal, W_regularizer = l2(reg_mf), input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = mf_dim, name = 'mf_embedding_item',
                                  init = init_normal, W_regularizer = l2(reg_mf), input_length=1)   """
    
    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = mf_dim, name = 'mf_embedding_user',
                                  embeddings_initializer = 'random_normal', W_regularizer = l2(reg_mf), input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = mf_dim, name = 'mf_embedding_item',
                                  embeddings_initializer = 'random_normal', W_regularizer = l2(reg_mf), input_length=1)  

    """MLP_Embedding_User = Embedding(input_dim = num_users, output_dim = layers[0]/2, name = "mlp_embedding_user",
                                  init = init_normal, W_regularizer = l2(reg_layers[0]), input_length=1)
    MLP_Embedding_Item = Embedding(input_dim = num_items, output_dim = layers[0]/2, name = 'mlp_embedding_item',
                                  init = init_normal, W_regularizer = l2(reg_layers[0]), input_length=1)  """ 
    
    MLP_Embedding_User = Embedding(input_dim = num_users, output_dim = int(layers[0]/2), name = "mlp_embedding_user",
                                   embeddings_initializer = 'random_normal', W_regularizer = l2(reg_layers[0]), input_length=1)
    MLP_Embedding_Item = Embedding(input_dim = num_items, output_dim = int(layers[0]/2), name = 'mlp_embedding_item',
                                   embeddings_initializer = 'random_normal', W_regularizer = l2(reg_layers[0]), input_length=1)

    output = MLP_Embedding_User(user_input)
    print(MLP_Embedding_User.weights)
    
    # MF part
    mf_user_latent = Flatten()(MF_Embedding_User(user_input))
    mf_item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    mf_vector = Multiply()([mf_user_latent, mf_item_latent]) # element-wise multiply

    # MLP part 
    mlp_user_latent = Flatten()(MLP_Embedding_User(user_input))
    mlp_item_latent = Flatten()(MLP_Embedding_Item(item_input))
    
    mlp_vector = Concatenate()([mlp_user_latent, mlp_item_latent])
    for idx in range(1, num_layer):
        layer = Dense(layers[idx], W_regularizer= l2(reg_layers[idx]), activation='relu', name="layer%d" %idx)
        mlp_vector = layer(mlp_vector)

    # Concatenate MF and MLP parts
    #mf_vector = Lambda(lambda x: x * alpha)(mf_vector)
    #mlp_vector = Lambda(lambda x : x * (1-alpha))(mlp_vector)
    predict_vector = Concatenate()([mf_vector, mlp_vector])
    
    # Final prediction layer
    prediction = Dense(1, activation='sigmoid', init='lecun_uniform', name = "prediction")(predict_vector)
    
    model = Model(input=[user_input, item_input], 
                  output=prediction)
    
    return model

def load_pretrain_model(model, gmf_model, mlp_model, num_layers):
    # MF embeddings
    gmf_user_embeddings = gmf_model.get_layer('user_embedding').get_weights()
    gmf_item_embeddings = gmf_model.get_layer('item_embedding').get_weights()
    model.get_layer('mf_embedding_user').set_weights(gmf_user_embeddings)
    model.get_layer('mf_embedding_item').set_weights(gmf_item_embeddings)
    
    # MLP embeddings
    mlp_user_embeddings = mlp_model.get_layer('user_embedding').get_weights()
    mlp_item_embeddings = mlp_model.get_layer('item_embedding').get_weights()
    model.get_layer('mlp_embedding_user').set_weights(mlp_user_embeddings)
    model.get_layer('mlp_embedding_item').set_weights(mlp_item_embeddings)
    
    # MLP layers
    for i in range(1, num_layers):
        mlp_layer_weights = mlp_model.get_layer('layer%d' %i).get_weights()
        model.get_layer('layer%d' %i).set_weights(mlp_layer_weights)
        
    # Prediction weights
    gmf_prediction = gmf_model.get_layer('prediction').get_weights()
    mlp_prediction = mlp_model.get_layer('prediction').get_weights()
    new_weights = np.concatenate((gmf_prediction[0], mlp_prediction[0]), axis=0)
    new_b = gmf_prediction[1] + mlp_prediction[1]
    model.get_layer('prediction').set_weights([0.5*new_weights, 0.5*new_b])    
    return model

def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    s1 = set(train.keys())
    l1 = len(train)
    i1=0; verb = 100;
    for (u, i) in train.keys():
        i1+=1;
        if i1 % verb == 0:
            print(i1)
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while (u, j) in s1:
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

Using TensorFlow backend.


In [6]:
sys.argv = ['NeuMF.py --dataset ml-1m --epochs 20 --batch_size 256 --num_factors 8 --layers [64,32,16,8] --num_neg 4 --lr 0.001 --learner adam --verbose 1 --out 1 --mf_pretrain Pretrain/ml-1m_GMF_8_1501651698.h5 --mlp_pretrain Pretrain/ml-1m_MLP_[64,32,16,8]_1501652038.h5']

In [7]:
args = parse_args()
num_epochs = args.epochs
batch_size = args.batch_size
mf_dim = args.num_factors
layers = eval(args.layers)
reg_mf = args.reg_mf
reg_layers = eval(args.reg_layers)
num_negatives = args.num_neg
learning_rate = args.lr
learner = args.learner
verbose = args.verbose
mf_pretrain = args.mf_pretrain
mlp_pretrain = args.mlp_pretrain

topK = 10
evaluation_threads = 1 # mp.cpu_count()
print("NeuMF arguments: %s " %(args))
model_out_file = 'Pretrain/%s_NeuMF_%d_%s_%d.h5' %(args.dataset, mf_dim, args.layers, time())

NeuMF arguments: Namespace(batch_size=256, dataset='ml-1m', epochs=100, layers='[64,32,16,8]', learner='adam', lr=0.001, mf_pretrain='', mlp_pretrain='', num_factors=8, num_neg=4, out=1, path='Data/', reg_layers='[0,0,0,0]', reg_mf=0, verbose=1) 


In [4]:
# Loading data
t1 = time()
dataset = Dataset(args.path + args.dataset)
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
num_users, num_items = train.shape
print("Load data done [%.1f s]. #user=%d, #item=%d, #train=%d, #test=%d" 
      %(time()-t1, num_users, num_items, train.nnz, len(testRatings)))

Load data done [14.3 s]. #user=6040, #item=3706, #train=994169, #test=6040


In [10]:
#user_input, item_input, labels = [],[],[]
#num_users = train.shape[0]
#s1 = set(train.keys())
#l1 = len(train)
i1=0; verb = 1;
for (u, i) in train.keys():
    i1+=1;
    if i1 % verb == 0:
        print((u, i), train[(u, i)])
#     # positive instance
#     user_input.append(u)
#     item_input.append(i)
#     labels.append(1)
#     # negative instances
#     for t in range(num_negatives):
#         j = np.random.randint(num_items)
#         while (u, j) in s1:
#             j = np.random.randint(num_items)
#         user_input.append(u)
#         item_input.append(j)
#         labels.append(0)

(0, 32) 1.0
(0, 34) 1.0
(0, 4) 1.0
(0, 35) 1.0
(0, 30) 1.0
(0, 29) 1.0
(0, 33) 1.0
(0, 40) 1.0
(0, 10) 1.0
(0, 16) 1.0
(0, 23) 1.0
(0, 28) 1.0
(0, 12) 1.0
(0, 8) 1.0
(0, 5) 1.0
(0, 20) 1.0
(0, 46) 1.0
(0, 15) 1.0
(0, 50) 1.0
(0, 49) 1.0
(0, 13) 1.0
(0, 1) 1.0
(0, 52) 1.0
(0, 38) 1.0
(0, 19) 1.0
(0, 6) 1.0
(0, 2) 1.0
(0, 26) 1.0
(0, 17) 1.0
(0, 39) 1.0
(0, 45) 1.0
(0, 42) 1.0
(0, 14) 1.0
(0, 11) 1.0
(0, 18) 1.0
(0, 48) 1.0
(0, 41) 1.0
(0, 43) 1.0
(0, 51) 1.0
(0, 9) 1.0
(0, 44) 1.0
(0, 21) 1.0
(0, 0) 1.0
(0, 47) 1.0
(0, 7) 1.0
(0, 3) 1.0
(0, 36) 1.0
(0, 24) 1.0
(0, 22) 1.0
(0, 37) 1.0
(0, 27) 1.0
(0, 31) 1.0
(1, 86) 1.0
(1, 66) 1.0
(1, 72) 1.0
(1, 130) 1.0
(1, 73) 1.0
(1, 79) 1.0
(1, 126) 1.0
(1, 163) 1.0
(1, 91) 1.0
(1, 98) 1.0
(1, 60) 1.0
(1, 137) 1.0
(1, 107) 1.0
(1, 118) 1.0
(1, 154) 1.0
(1, 146) 1.0
(1, 69) 1.0
(1, 74) 1.0
(1, 83) 1.0
(1, 160) 1.0
(1, 157) 1.0
(1, 94) 1.0
(1, 65) 1.0
(1, 125) 1.0
(1, 119) 1.0
(1, 58) 1.0
(1, 122) 1.0
(1, 117) 1.0
(1, 93) 1.0
(1, 120) 1.0
(1, 166) 1.

(9, 179) 1.0
(9, 736) 1.0
(9, 615) 1.0
(9, 288) 1.0
(9, 605) 1.0
(9, 731) 1.0
(9, 650) 1.0
(9, 673) 1.0
(9, 6) 1.0
(9, 292) 1.0
(9, 20) 1.0
(9, 534) 1.0
(9, 779) 1.0
(9, 419) 1.0
(9, 377) 1.0
(9, 563) 1.0
(9, 587) 1.0
(9, 681) 1.0
(9, 729) 1.0
(9, 322) 1.0
(9, 645) 1.0
(9, 649) 1.0
(9, 725) 1.0
(9, 110) 1.0
(9, 728) 1.0
(9, 684) 1.0
(9, 89) 1.0
(9, 553) 1.0
(9, 312) 1.0
(9, 571) 1.0
(9, 187) 1.0
(9, 704) 1.0
(9, 153) 1.0
(9, 567) 1.0
(9, 778) 1.0
(9, 7) 1.0
(9, 83) 1.0
(9, 766) 1.0
(9, 708) 1.0
(9, 8) 1.0
(9, 685) 1.0
(9, 283) 1.0
(9, 77) 1.0
(9, 677) 1.0
(9, 748) 1.0
(9, 608) 1.0
(9, 112) 1.0
(9, 171) 1.0
(9, 414) 1.0
(9, 664) 1.0
(9, 2) 1.0
(9, 618) 1.0
(9, 158) 1.0
(9, 424) 1.0
(9, 111) 1.0
(9, 390) 1.0
(9, 236) 1.0
(9, 547) 1.0
(9, 4) 1.0
(9, 585) 1.0
(9, 692) 1.0
(9, 601) 1.0
(9, 40) 1.0
(9, 693) 1.0
(9, 696) 1.0
(9, 581) 1.0
(9, 149) 1.0
(9, 189) 1.0
(9, 552) 1.0
(9, 761) 1.0
(9, 593) 1.0
(9, 711) 1.0
(9, 39) 1.0
(9, 438) 1.0
(9, 505) 1.0
(9, 462) 1.0
(9, 203) 1.0
(9, 560) 1.0
(9

(17, 17) 1.0
(17, 923) 1.0
(17, 468) 1.0
(17, 682) 1.0
(17, 1046) 1.0
(17, 8) 1.0
(17, 10) 1.0
(17, 588) 1.0
(17, 596) 1.0
(17, 620) 1.0
(17, 389) 1.0
(17, 609) 1.0
(17, 531) 1.0
(17, 390) 1.0
(17, 608) 1.0
(17, 4) 1.0
(17, 1027) 1.0
(17, 580) 1.0
(17, 33) 1.0
(17, 40) 1.0
(17, 1095) 1.0
(17, 1081) 1.0
(17, 180) 1.0
(17, 191) 1.0
(17, 287) 1.0
(17, 126) 1.0
(17, 125) 1.0
(17, 1036) 1.0
(17, 857) 1.0
(17, 577) 1.0
(17, 1025) 1.0
(17, 200) 1.0
(17, 858) 1.0
(17, 846) 1.0
(17, 1016) 1.0
(17, 845) 1.0
(17, 733) 1.0
(17, 1080) 1.0
(17, 842) 1.0
(17, 123) 1.0
(17, 884) 1.0
(17, 84) 1.0
(17, 9) 1.0
(17, 5) 1.0
(17, 364) 1.0
(17, 467) 1.0
(17, 1039) 1.0
(17, 218) 1.0
(17, 246) 1.0
(17, 1044) 1.0
(17, 1026) 1.0
(17, 1094) 1.0
(17, 1084) 1.0
(17, 1063) 1.0
(17, 1085) 1.0
(17, 1040) 1.0
(17, 1055) 1.0
(17, 1082) 1.0
(17, 929) 1.0
(17, 1029) 1.0
(17, 1067) 1.0
(17, 1103) 1.0
(17, 569) 1.0
(17, 1074) 1.0
(17, 1073) 1.0
(17, 1066) 1.0
(17, 1104) 1.0
(17, 1053) 1.0
(17, 1072) 1.0
(17, 1089) 1.0
(17, 

(22, 922) 1.0
(22, 272) 1.0
(22, 982) 1.0
(22, 255) 1.0
(22, 59) 1.0
(22, 1107) 1.0
(22, 1231) 1.0
(22, 945) 1.0
(22, 1272) 1.0
(22, 1286) 1.0
(22, 1247) 1.0
(22, 229) 1.0
(22, 336) 1.0
(22, 497) 1.0
(22, 770) 1.0
(22, 136) 1.0
(22, 246) 1.0
(22, 67) 1.0
(22, 1237) 1.0
(22, 1249) 1.0
(22, 43) 1.0
(22, 1294) 1.0
(22, 434) 1.0
(22, 51) 1.0
(22, 244) 1.0
(22, 669) 1.0
(22, 259) 1.0
(22, 1066) 1.0
(22, 1026) 1.0
(22, 808) 1.0
(22, 656) 1.0
(22, 673) 1.0
(22, 678) 1.0
(22, 852) 1.0
(22, 754) 1.0
(22, 573) 1.0
(22, 558) 1.0
(22, 591) 1.0
(22, 606) 1.0
(22, 636) 1.0
(22, 717) 1.0
(22, 758) 1.0
(22, 767) 1.0
(22, 642) 1.0
(22, 19) 1.0
(22, 424) 1.0
(22, 842) 1.0
(22, 655) 1.0
(22, 514) 1.0
(22, 44) 1.0
(22, 741) 1.0
(22, 26) 1.0
(22, 720) 1.0
(22, 1293) 1.0
(22, 1244) 1.0
(22, 1257) 1.0
(22, 58) 1.0
(22, 909) 1.0
(22, 1) 1.0
(22, 1254) 1.0
(22, 603) 1.0
(22, 1236) 1.0
(22, 380) 1.0
(22, 331) 1.0
(22, 237) 1.0
(22, 349) 1.0
(22, 1131) 1.0
(22, 379) 1.0
(22, 242) 1.0
(22, 728) 1.0
(22, 531) 1.0


(29, 256) 1.0
(29, 415) 1.0
(29, 1483) 1.0
(29, 21) 1.0
(29, 259) 1.0
(29, 164) 1.0
(29, 246) 1.0
(29, 1488) 1.0
(29, 104) 1.0
(29, 244) 1.0
(29, 67) 1.0
(29, 128) 1.0
(29, 48) 1.0
(29, 28) 1.0
(29, 23) 1.0
(29, 452) 1.0
(29, 1486) 1.0
(29, 167) 1.0
(29, 417) 1.0
(29, 1105) 1.0
(29, 1484) 1.0
(29, 26) 1.0
(29, 1490) 1.0
(29, 797) 1.0
(29, 1142) 1.0
(29, 392) 1.0
(30, 1181) 1.0
(30, 1264) 1.0
(30, 399) 1.0
(30, 601) 1.0
(30, 731) 1.0
(30, 1518) 1.0
(30, 1508) 1.0
(30, 83) 1.0
(30, 614) 1.0
(30, 832) 1.0
(30, 1460) 1.0
(30, 695) 1.0
(30, 1511) 1.0
(30, 777) 1.0
(30, 534) 1.0
(30, 676) 1.0
(30, 1502) 1.0
(30, 1351) 1.0
(30, 698) 1.0
(30, 1495) 1.0
(30, 1515) 1.0
(30, 1506) 1.0
(30, 1001) 1.0
(30, 1050) 1.0
(30, 1492) 1.0
(30, 730) 1.0
(30, 1256) 1.0
(30, 565) 1.0
(30, 1504) 1.0
(30, 1491) 1.0
(30, 1503) 1.0
(30, 1519) 1.0
(30, 1476) 1.0
(30, 517) 1.0
(30, 145) 1.0
(30, 933) 1.0
(30, 792) 1.0
(30, 409) 1.0
(30, 625) 1.0
(30, 521) 1.0
(30, 203) 1.0
(30, 1496) 1.0
(30, 814) 1.0
(30, 1280) 1.

...

(732, 3308) 1.0
(732, 1753) 1.0
(732, 1120) 1.0
(732, 890) 1.0
(732, 756) 1.0
(732, 1344) 1.0
(732, 633) 1.0
(732, 275) 1.0
(732, 1913) 1.0
(732, 1061) 1.0
(732, 1297) 1.0
(732, 88) 1.0
(732, 1529) 1.0
(732, 2489) 1.0
(732, 184) 1.0
(732, 206) 1.0
(732, 5) 1.0
(732, 705) 1.0
(732, 535) 1.0
(732, 29) 1.0
(732, 337) 1.0
(732, 55) 1.0
(732, 795) 1.0
(732, 106) 1.0
(732, 177) 1.0
(732, 1517) 1.0
(732, 797) 1.0
(732, 134) 1.0
(732, 0) 1.0
(732, 194) 1.0
(732, 276) 1.0
(732, 529) 1.0
(732, 1292) 1.0
(732, 1520) 1.0
(732, 41) 1.0
(732, 559) 1.0
(732, 48) 1.0
(732, 662) 1.0
(732, 414) 1.0
(732, 547) 1.0
(732, 1240) 1.0
(732, 244) 1.0
(732, 315) 1.0
(732, 246) 1.0
(732, 259) 1.0
(732, 1488) 1.0
(732, 483) 1.0
(732, 383) 1.0
(732, 104) 1.0
(732, 23) 1.0
(732, 167) 1.0
(732, 128) 1.0
(732, 978) 1.0
(732, 2555) 1.0
(732, 1058) 1.0
(732, 2139) 1.0
(732, 740) 1.0
(732, 762) 1.0
(732, 322) 1.0
(732, 775) 1.0
(732, 359) 1.0
(732, 376) 1.0
(732, 62) 1.0
(732, 60) 1.0
(732, 155) 1.0
(732, 220) 1.0
(732,

(738, 50) 1.0
(738, 168) 1.0
(738, 1271) 1.0
(738, 27) 1.0
(738, 799) 1.0
(738, 454) 1.0
(739, 340) 1.0
(739, 58) 1.0
(739, 452) 1.0
(739, 1110) 1.0
(739, 406) 1.0
(739, 544) 1.0
(739, 68) 1.0
(739, 1712) 1.0
(739, 519) 1.0
(739, 226) 1.0
(739, 2279) 1.0
(739, 113) 1.0
(739, 993) 1.0
(739, 1) 1.0
(739, 192) 1.0
(739, 1270) 1.0
(739, 509) 1.0
(739, 948) 1.0
(739, 2318) 1.0
(739, 15) 1.0
(739, 97) 1.0
(739, 283) 1.0
(739, 609) 1.0
(739, 499) 1.0
(739, 321) 1.0
(739, 1231) 1.0
(739, 2131) 1.0
(739, 104) 1.0
(739, 831) 1.0
(739, 1105) 1.0
(739, 437) 1.0
(739, 92) 1.0
(739, 447) 1.0
(739, 60) 1.0
(739, 151) 1.0
(739, 354) 1.0
(740, 1139) 1.0
(740, 2167) 1.0
(740, 2020) 1.0
(740, 1819) 1.0
(740, 2644) 1.0
(740, 1505) 1.0
(740, 1909) 1.0
(740, 1816) 1.0
(740, 1743) 1.0
(740, 1679) 1.0
(740, 2063) 1.0
(740, 1734) 1.0
(740, 1175) 1.0
(740, 1736) 1.0
(740, 1142) 1.0
(740, 2353) 1.0
(740, 910) 1.0
(740, 1814) 1.0
(740, 1813) 1.0
(740, 1931) 1.0
(740, 2259) 1.0
(740, 1811) 1.0
(740, 131) 1.0
(740,

(746, 132) 1.0
(746, 124) 1.0
(746, 44) 1.0
(746, 217) 1.0
(746, 2444) 1.0
(746, 480) 1.0
(746, 885) 1.0
(746, 340) 1.0
(746, 349) 1.0
(746, 519) 1.0
(746, 68) 1.0
(746, 2349) 1.0
(746, 499) 1.0
(746, 1040) 1.0
(746, 1485) 1.0
(746, 831) 1.0
(746, 541) 1.0
(746, 2060) 1.0
(746, 2172) 1.0
(746, 2636) 1.0
(746, 48) 1.0
(746, 155) 1.0
(747, 1065) 1.0
(747, 241) 1.0
(747, 1812) 1.0
(747, 1811) 1.0
(747, 1808) 1.0
(747, 1813) 1.0
(747, 764) 1.0
(747, 1810) 1.0
(747, 1993) 1.0
(747, 768) 1.0
(747, 463) 1.0
(747, 505) 1.0
(747, 2003) 1.0
(747, 669) 1.0
(747, 29) 1.0
(747, 591) 1.0
(747, 432) 1.0
(747, 251) 1.0
(747, 528) 1.0
(747, 155) 1.0
(747, 702) 1.0
(747, 1018) 1.0
(747, 615) 1.0
(747, 26) 1.0
(747, 599) 1.0
(747, 62) 1.0
(747, 64) 1.0
(747, 376) 1.0
(747, 132) 1.0
(747, 740) 1.0
(747, 758) 1.0
(747, 97) 1.0
(747, 22) 1.0
(747, 756) 1.0
(747, 1466) 1.0
(747, 216) 1.0
(747, 217) 1.0
(747, 206) 1.0
(747, 562) 1.0
(747, 434) 1.0
(747, 51) 1.0
(747, 9) 1.0
(747, 338) 1.0
(747, 0) 1.0
(747, 1

(751, 392) 1.0
(751, 1210) 1.0
(751, 182) 1.0
(751, 15) 1.0
(751, 547) 1.0
(751, 177) 1.0
(751, 5) 1.0
(751, 1579) 1.0
(751, 603) 1.0
(751, 2494) 1.0
(751, 175) 1.0
(751, 206) 1.0
(751, 1517) 1.0
(751, 2167) 1.0
(751, 1013) 1.0
(751, 528) 1.0
(751, 113) 1.0
(751, 189) 1.0
(751, 364) 1.0
(751, 220) 1.0
(751, 1287) 1.0
(751, 1305) 1.0
(751, 1717) 1.0
(751, 914) 1.0
(751, 3188) 1.0
(751, 2082) 1.0
(751, 1723) 1.0
(751, 2962) 1.0
(751, 73) 1.0
(751, 1883) 1.0
(751, 1190) 1.0
(751, 2135) 1.0
(751, 1711) 1.0
(751, 1797) 1.0
(751, 2392) 1.0
(751, 1172) 1.0
(751, 564) 1.0
(751, 126) 1.0
(751, 2099) 1.0
(751, 1192) 1.0
(751, 66) 1.0
(751, 86) 1.0
(751, 1145) 1.0
(751, 1204) 1.0
(751, 1155) 1.0
(751, 125) 1.0
(751, 1670) 1.0
(751, 883) 1.0
(751, 1901) 1.0
(751, 849) 1.0
(751, 1829) 1.0
(751, 2144) 1.0
(751, 2899) 1.0
(751, 222) 1.0
(751, 130) 1.0
(751, 843) 1.0
(751, 1926) 1.0
(751, 202) 1.0
(751, 925) 1.0
(751, 1419) 

KeyboardInterrupt: 

In [5]:
print(num_users, num_items, mf_dim, layers, reg_layers, reg_mf)
print(type(layers[1]))

6040 3706 8 [64, 32, 16, 8] [0, 0, 0, 0] 0
<class 'int'>


In [6]:
# Build model
model = get_model(num_users, num_items, mf_dim, layers, reg_layers, reg_mf)

Input layer:  (None, 1) <dtype: 'int32'>
[<tf.Variable 'mlp_embedding_user/embeddings:0' shape=(6040, 32) dtype=float32, numpy=
array([[-0.10085068,  0.10213051,  0.02510002, ...,  0.04082659,
         0.07254616, -0.00987088],
       [ 0.022273  , -0.06715929,  0.03057785, ...,  0.06460108,
        -0.01856003, -0.06164598],
       [-0.0181519 ,  0.00563864,  0.01537797, ...,  0.01842873,
        -0.00103753, -0.02682051],
       ...,
       [-0.01473483, -0.00513661, -0.02475733, ..., -0.01454086,
        -0.04368399,  0.147334  ],
       [-0.03013107, -0.03938418,  0.00162018, ..., -0.03639964,
         0.06655296,  0.05786943],
       [-0.04506666, -0.04215284,  0.01508992, ..., -0.10154399,
        -0.03797027, -0.00530705]], dtype=float32)>]


NeuMF.py --dataset ml-1m --epochs 20 --batch_size 256 --num_factors 8 --layers [64,32,16,8] --num_neg 4 --lr 0.001 --learner adam --verbose 1 --out 1 --mf_pretrain Pretrain/ml-1m_GMF_8_1501651698.h5 --mlp_pretrain Pretrain/ml-1m_MLP_[64,32,16,8]_1501652038.h5:87: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(input_dim=6040, output_dim=8, name="mf_embedding_user", embeddings_initializer="random_normal", input_length=1, embeddings_regularizer=<keras.reg...)`
NeuMF.py --dataset ml-1m --epochs 20 --batch_size 256 --num_factors 8 --layers [64,32,16,8] --num_neg 4 --lr 0.001 --learner adam --verbose 1 --out 1 --mf_pretrain Pretrain/ml-1m_GMF_8_1501651698.h5 --mlp_pretrain Pretrain/ml-1m_MLP_[64,32,16,8]_1501652038.h5:89: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(input_dim=3706, output_dim=8, name="mf_embedding_item", embeddings_initializer="random_normal", input_length=1, embeddings_regularizer=<keras.reg...)`
NeuMF.py --dataset ml-1m

In [7]:
if learner.lower() == "adagrad": 
    model.compile(optimizer=Adagrad(lr=learning_rate), loss='binary_crossentropy')
elif learner.lower() == "rmsprop":
    model.compile(optimizer=RMSprop(lr=learning_rate), loss='binary_crossentropy')
elif learner.lower() == "adam":
    model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy')
else:
    model.compile(optimizer=SGD(lr=learning_rate), loss='binary_crossentropy')

In [8]:
# Load pretrain model
if mf_pretrain != '' and mlp_pretrain != '':
    gmf_model = GMF.get_model(num_users,num_items,mf_dim)
    gmf_model.load_weights(mf_pretrain)
    mlp_model = MLP.get_model(num_users,num_items, layers, reg_layers)
    mlp_model.load_weights(mlp_pretrain)
    model = load_pretrain_model(model, gmf_model, mlp_model, len(layers))
    print("Load pretrained GMF (%s) and MLP (%s) models done. " %(mf_pretrain, mlp_pretrain))

In [9]:
# Init performance
(hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
print('Init: HR = %.4f, NDCG = %.4f' % (hr, ndcg))
best_hr, best_ndcg, best_iter = hr, ndcg, -1
if args.out > 0:
    model.save_weights(model_out_file, overwrite=True) 

Init: HR = 0.1022, NDCG = 0.0473


In [ ]:
# Training model
for epoch in range(num_epochs):
    t1 = time()
    # Generate training instances
    user_input, item_input, labels = get_train_instances(train, num_negatives)
    print('Finish generating')
    
    # Training
    hist = model.fit([np.array(user_input), np.array(item_input)], #input
                     np.array(labels), # labels 
                     batch_size=batch_size, nb_epoch=1, verbose=0, shuffle=True)
    t2 = time()

    # Evaluation
    if epoch % verbose == 0:
        (hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
        hr, ndcg, loss = np.array(hits).mean(), np.array(ndcgs).mean(), hist.history['loss'][0]
        print('Iteration %d [%.1f s]: HR = %.4f, NDCG = %.4f, loss = %.4f [%.1f s]' 
              % (epoch,  t2-t1, hr, ndcg, loss, time()-t2))
        if hr > best_hr:
            best_hr, best_ndcg, best_iter = hr, ndcg, epoch
            if args.out > 0:
                model.save_weights(model_out_file, overwrite=True)

print("End. Best Iteration %d:  HR = %.4f, NDCG = %.4f. " %(best_iter, best_hr, best_ndcg))
if args.out > 0:
    print("The best NeuMF model is saved to %s" %(model_out_file))

100000
200000
300000
400000
500000
600000
700000
800000
900000
Finish generating


NeuMF.py --dataset ml-1m --epochs 20 --batch_size 256 --num_factors 8 --layers [64,32,16,8] --num_neg 4 --lr 0.001 --learner adam --verbose 1 --out 1 --mf_pretrain Pretrain/ml-1m_GMF_8_1501651698.h5 --mlp_pretrain Pretrain/ml-1m_MLP_[64,32,16,8]_1501652038.h5:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
D:\Anaconda3\envs\CS412-project\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Iteration 0 [69.4 s]: HR = 0.5843, NDCG = 0.3288, loss = 0.3246 [4.4 s]
100000
200000
300000
400000
500000
600000
700000
800000
900000
Finish generating


NeuMF.py --dataset ml-1m --epochs 20 --batch_size 256 --num_factors 8 --layers [64,32,16,8] --num_neg 4 --lr 0.001 --learner adam --verbose 1 --out 1 --mf_pretrain Pretrain/ml-1m_GMF_8_1501651698.h5 --mlp_pretrain Pretrain/ml-1m_MLP_[64,32,16,8]_1501652038.h5:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Iteration 1 [72.0 s]: HR = 0.6293, NDCG = 0.3644, loss = 0.2770 [4.1 s]
100000
200000
300000
400000
500000
600000
700000
800000
900000
Finish generating
Iteration 2 [73.5 s]: HR = 0.6533, NDCG = 0.3812, loss = 0.2643 [4.3 s]
100000
200000
300000
400000
500000
600000
700000
800000
900000
Finish generating
Iteration 3 [76.5 s]: HR = 0.6606, NDCG = 0.3908, loss = 0.2581 [4.9 s]
100000
200000
300000
400000
500000
600000
700000
800000
900000
Finish generating
Iteration 4 [78.2 s]: HR = 0.6656, NDCG = 0.3918, loss = 0.2542 [4.9 s]
100000
200000
300000
400000
500000
600000
700000
800000
900000
Finish generating
Iteration 5 [80.5 s]: HR = 0.6690, NDCG = 0.3981, loss = 0.2510 [4.8 s]
100000
200000
300000
400000
500000
600000
700000
800000
900000
Finish generating
Iteration 6 [83.8 s]: HR = 0.6687, NDCG = 0.3958, loss = 0.2483 [5.2 s]
100000
200000
300000
400000
500000
600000
700000
800000
900000
Finish generating
Iteration 7 [86.4 s]: HR = 0.6765, NDCG = 0.4021, loss = 0.2457 [7.0 s]
100000
200